<a href="https://colab.research.google.com/github/Annie-Yeeun-Jang/Info_retrieval_project/blob/master/%5BSI650_project%5DMatrix_Factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse.linalg import svds

from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir('/content/gdrive/MyDrive/650/team_proj')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install --upgrade xlrd

In [ ]:
jester=pd.read_excel('./data/FINAL jester 2006-15.xls', header=None)

In [ ]:
jester

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,62,99,99,99,99,0.21875,99,-9.28125,-9.28125,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
1,34,99,99,99,99,-9.68750,99,9.93750,9.53125,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
2,18,99,99,99,99,-9.84375,99,-9.84375,-7.21875,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
3,82,99,99,99,99,6.90625,99,4.75000,-5.90625,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
4,27,99,99,99,99,-0.03125,99,-9.09375,-0.40625,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54900,13,99,99,99,99,99.00000,99,-6.53125,-2.34375,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
54901,8,99,99,99,99,99.00000,99,8.93750,9.78125,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
54902,27,99,99,99,99,99.00000,99,-1.59375,4.53125,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
54903,8,99,99,99,99,99.00000,99,-7.40625,6.93750,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0


In [ ]:
jester.isnull().sum().sum()

28

In [ ]:
jester.fillna(99, inplace=True)

In [ ]:
# MF
ply_pivot1 = ply_pivot1.pivot_table("link",index="id",columns="song_id")
ply_pivot1.fillna(0,inplace=True)
ply_pivot1

In [ ]:
# matrix는 pivot_table 값을 numpy matrix로 만든 것 
matrix = ply_pivot1.values
# user_link_mean은 사용자의 평균 연결 
user_link_mean = np.mean(matrix, axis = 1)
# R_user_mean : 사용자-노래에 대해 사용자 평균 연결을 뺀 것.
matrix_user_mean = matrix - user_link_mean.reshape(-1, 1)

In [ ]:
# scipy에서 제공해주는 svd.  
# U 행렬, sigma 행렬, V 전치 행렬을 반환.
# https://seing.tistory.com/67
U, sigma, Vt = svds(jester, k = 6)
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(54905, 6)
(6,)
(6, 151)


In [ ]:
sigma = np.diag(sigma)
sigma[0]

array([12891.45673842,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ])

In [ ]:
# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다.
# 거기에 + 사용자 평균 rating을 적용한다.
svd_user_predicted_link = np.dot(np.dot(U, sigma), Vt) #+ user_link_mean.reshape(-1, 1)

In [ ]:
pd.DataFrame(svd_user_predicted_link)

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,10.719952,66.230494,66.230494,66.230494,66.230494,66.003736,66.230494,-1.942718,-1.057007,66.230494,...,87.793610,61.206502,45.946603,65.937696,52.362757,86.480039,65.177066,34.804627,53.966494,51.863223
1,0.601387,77.580292,77.580292,77.580292,77.580292,76.830063,77.580292,-1.726755,-0.865600,77.580292,...,96.732098,85.721013,76.433212,86.682802,81.302885,96.900274,86.888192,66.314287,80.309819,66.360219
2,6.358041,91.007476,91.007476,91.007476,91.007476,89.443765,91.007476,-1.688598,-0.727612,91.007476,...,86.478227,92.727868,92.337974,91.293487,94.662776,86.745976,93.072740,100.685981,98.608622,102.760407
3,24.467622,62.245877,62.245877,62.245877,62.245877,60.348848,62.245877,-0.065362,0.300435,62.245877,...,52.123462,60.386524,73.924908,47.070411,74.896311,51.321315,47.045412,43.447713,47.831208,49.786623
4,-12.837699,72.743751,72.743751,72.743751,72.743751,71.769862,72.743751,-1.692091,-0.944605,72.743751,...,96.895540,85.575648,86.410357,91.010463,90.529945,97.562064,91.724554,74.629548,85.733141,86.244298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54900,12.848062,98.821247,98.821247,98.821247,98.821247,97.408081,98.821247,-2.523860,-1.322306,98.821247,...,100.372829,102.420856,68.426354,94.869678,75.606213,99.720021,96.728217,98.948979,103.272779,106.504926
54901,9.040219,99.639722,99.639722,99.639722,99.639722,98.217157,99.639722,-1.870113,-0.777976,99.639722,...,96.938690,98.123320,100.425620,99.420593,102.778346,97.114130,100.860735,102.938821,103.463747,102.595417
54902,23.930360,96.710203,96.710203,96.710203,96.710203,95.195460,96.710203,-1.494396,-0.588894,96.710203,...,111.877962,111.739259,79.111049,90.922149,87.542715,111.185352,90.541691,61.688352,85.176434,50.057699
54903,9.051901,99.651292,99.651292,99.651292,99.651292,98.230435,99.651292,-1.873133,-0.780038,99.651292,...,96.953611,98.123681,100.333278,99.421798,102.696261,97.127016,100.863152,102.946830,103.474358,102.605886


In [ ]:
jester

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,62,99,99,99,99,0.21875,99,-9.28125,-9.28125,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
1,34,99,99,99,99,-9.68750,99,9.93750,9.53125,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
2,18,99,99,99,99,-9.84375,99,-9.84375,-7.21875,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
3,82,99,99,99,99,6.90625,99,4.75000,-5.90625,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
4,27,99,99,99,99,-0.03125,99,-9.09375,-0.40625,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54900,13,99,99,99,99,99.00000,99,-6.53125,-2.34375,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
54901,8,99,99,99,99,99.00000,99,8.93750,9.78125,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
54902,27,99,99,99,99,99.00000,99,-1.59375,4.53125,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
54903,8,99,99,99,99,99.00000,99,-7.40625,6.93750,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0


## Below here is just for reference
- Codes we can refer for more advanced models:
- https://github.com/microsoft/recommenders
- https://github.com/microsoft/recommenders/blob/main/examples/02_model_collaborative_filtering/lightgcn_deep_dive.ipynb
- https://github.com/microsoft/recommenders/blob/main/examples/02_model_collaborative_filtering/standard_vae_deep_dive.ipynb


In [ ]:
df_svd_preds = pd.DataFrame(svd_user_predicted_link,index= ply_pivot1.index ,columns = ply_pivot1.columns)
df_svd_preds

In [ ]:
def recommend_songs(df_svd_preds, user_id, ori_link_df, num_recommendations):
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 곡 데이터 정렬 -> 영화 유사도가 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[user_id].sort_values(ascending=False)
    # 원본 곡 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    #user_data = ori_link_df.loc[user_id]
    # 사용자의 곡 유사도가 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = pd.DataFrame(sorted_user_predictions).reset_index()
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_id: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
    return recommendations

In [ ]:
predictions = recommend_songs(df_svd_preds, 23431, ply_pivot1, 100)

In [ ]:
predictions

In [ ]:
import json
 
# Ready for data
songs = []
song = {}
for name in ply_pivot_val.id:
  song['id'] = name
  song['songs'] = list(recommend_songs(df_svd_preds, name, ply_pivot1, 100)['song_id'])
  songs.append(song)

 
# Print JSON
print(json.dumps(songs, indent="\t") )

In [ ]:
# Write JSON
with open('./답안/res.json', 'w', encoding="utf-8") as make_file:
    json.dump(songs, make_file, ensure_ascii=False, indent="\t")

In [ ]:
f = open('./답안/res.json', encoding="UTF-8")
raw_data = json.loads(f.read())
raw_data